# HAR CNN training 

In [1]:
# Imports
import numpy as np
import os
from utils.semg_utilities import *
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

## Prepare data

In [2]:
X_train, labels_train, list_ch_train = read_data(data_path="C:\\Users\\Daniel\\Desktop\\GSU 2018\\Machine Learning\\sEMG-Neural-Net\\sEMG\\Database 2\\male_day_1\\", split="train") # train
X_test, labels_test, list_ch_test = read_data(data_path="C:\\Users\\Daniel\\Desktop\\GSU 2018\\Machine Learning\\sEMG-Neural-Net\\sEMG\\Database 2\\male_day_2\\", split="test") # test

assert list_ch_train == list_ch_test, "Mistmatch in channels!"

In [3]:
# Normalize?
X_train, X_test = standardize(X_train, X_test)

Train/Validation Split

In [4]:
X_tr, X_vld, lab_tr, lab_vld = train_test_split(X_train, labels_train, 
                                                stratify = labels_train, random_state = 123)

One-hot encoding:

In [5]:
y_tr = one_hot(lab_tr)
y_vld = one_hot(lab_vld)
y_test = one_hot(labels_test)

In [6]:
# Imports
import tensorflow as tf

### Hyperparameters

In [7]:
batch_size = 25       # Batch size
seq_len = 2500          # Number of steps
learning_rate = 0.0001
epochs = 1000

n_classes = 6
n_channels = 2

### Construct the graph
Placeholders

In [8]:
graph = tf.Graph()

# Construct placeholders
with graph.as_default():
    inputs_ = tf.placeholder(tf.float32, [None, seq_len, n_channels], name = 'inputs')
    labels_ = tf.placeholder(tf.float32, [None, n_classes], name = 'labels')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')

Build Convolutional Layers

Note: Should we use a different activation? Like tf.nn.tanh?

In [9]:
with graph.as_default():
    # (batch, 2500, 2) --> (batch, 1250, 4)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=4, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=2, strides=2, padding='same')
    
    # (batch, 1250, 4) --> (batch, 625, 8)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=8, kernel_size=2, strides=1, 
                             padding='same', activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=2, padding='same')
    
    # (batch, 32, 36) --> (batch, 16, 72)
    # conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=16, kernel_size=2, strides=1, 
    #                          padding='same', activation = tf.nn.relu)
    # max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=2, strides=2, padding='same')
    
    # (batch, 16, 72) --> (batch, 8, 144)
    # conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=32, kernel_size=2, strides=1, 
    #                          padding='same', activation = tf.nn.relu)
    # max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=2, padding='same')

Now, flatten and pass to the classifier

In [10]:
with graph.as_default():
    # Flatten and add dropout
    flat = tf.reshape(max_pool_2, (-1, 8 * 625))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)
    
    # Predictions
    logits = tf.layers.dense(flat, n_classes)
    
    # Cost function and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels_))
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)
    
    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(labels_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

### Train the network

In [11]:
if (os.path.exists('checkpoints-cnn') == False):
    !mkdir checkpoints-cnn

In [ ]:
validation_acc = []
validation_loss = []

train_acc = []
train_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    iteration = 1
   
    # Loop over epochs
    for e in range(epochs):
        
        # Loop over batches
        for x,y in get_batches(X_tr, y_tr, batch_size):
            
            # Feed dictionary
            feed = {inputs_ : x, labels_ : y, keep_prob_ : 0.5, learning_rate_ : learning_rate}
            
            # Loss
            loss, _ , acc = sess.run([cost, optimizer, accuracy], feed_dict = feed)
            train_acc.append(acc)
            train_loss.append(loss)
            
            # Print at each 5 iters
            if (iteration % 5 == 0):
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Train loss: {:6f}".format(loss),
                      "Train acc: {:.6f}".format(acc))
            
            # Compute validation loss at every 10 iterations
            if (iteration%10 == 0):                
                val_acc_ = []
                val_loss_ = []
                
                for x_v, y_v in get_batches(X_vld, y_vld, batch_size):
                    # Feed
                    feed = {inputs_ : x_v, labels_ : y_v, keep_prob_ : 1.0}  
                    
                    # Loss
                    loss_v, acc_v = sess.run([cost, accuracy], feed_dict = feed)                    
                    val_acc_.append(acc_v)
                    val_loss_.append(loss_v)
                
                # Print info
                print("Epoch: {}/{}".format(e, epochs),
                      "Iteration: {:d}".format(iteration),
                      "Validation loss: {:6f}".format(np.mean(val_loss_)),
                      "Validation acc: {:.6f}".format(np.mean(val_acc_)))
                
                # Store
                validation_acc.append(np.mean(val_acc_))
                validation_loss.append(np.mean(val_loss_))
            
            # Iterate 
            iteration += 1
    
    saver.save(sess,"checkpoints-cnn/har.ckpt")

Epoch: 0/1000 Iteration: 5 Train loss: 2.112994 Train acc: 0.160000
Epoch: 0/1000 Iteration: 10 Train loss: 2.066516 Train acc: 0.160000
Epoch: 0/1000 Iteration: 10 Validation loss: 1.793543 Validation acc: 0.226667
Epoch: 0/1000 Iteration: 15 Train loss: 2.099262 Train acc: 0.120000
Epoch: 1/1000 Iteration: 20 Train loss: 1.691031 Train acc: 0.200000
Epoch: 1/1000 Iteration: 20 Validation loss: 1.757354 Validation acc: 0.253333
Epoch: 1/1000 Iteration: 25 Train loss: 1.688824 Train acc: 0.280000
Epoch: 1/1000 Iteration: 30 Train loss: 1.911831 Train acc: 0.200000
Epoch: 1/1000 Iteration: 30 Validation loss: 1.731476 Validation acc: 0.253333
Epoch: 1/1000 Iteration: 35 Train loss: 1.654397 Train acc: 0.320000
Epoch: 2/1000 Iteration: 40 Train loss: 1.992802 Train acc: 0.240000
Epoch: 2/1000 Iteration: 40 Validation loss: 1.696666 Validation acc: 0.280000
Epoch: 2/1000 Iteration: 45 Train loss: 1.865819 Train acc: 0.320000
Epoch: 2/1000 Iteration: 50 Train loss: 1.872642 Train acc: 0.20

Epoch: 21/1000 Iteration: 380 Train loss: 0.779526 Train acc: 0.640000
Epoch: 21/1000 Iteration: 380 Validation loss: 1.197827 Validation acc: 0.400000
Epoch: 21/1000 Iteration: 385 Train loss: 0.770466 Train acc: 0.720000
Epoch: 21/1000 Iteration: 390 Train loss: 0.865811 Train acc: 0.760000
Epoch: 21/1000 Iteration: 390 Validation loss: 1.195910 Validation acc: 0.406667
Epoch: 21/1000 Iteration: 395 Train loss: 0.959473 Train acc: 0.600000
Epoch: 22/1000 Iteration: 400 Train loss: 1.032077 Train acc: 0.480000
Epoch: 22/1000 Iteration: 400 Validation loss: 1.191960 Validation acc: 0.400000
Epoch: 22/1000 Iteration: 405 Train loss: 0.929688 Train acc: 0.640000
Epoch: 22/1000 Iteration: 410 Train loss: 0.825037 Train acc: 0.720000
Epoch: 22/1000 Iteration: 410 Validation loss: 1.188151 Validation acc: 0.400000
Epoch: 23/1000 Iteration: 415 Train loss: 1.070449 Train acc: 0.520000
Epoch: 23/1000 Iteration: 420 Train loss: 0.791797 Train acc: 0.800000
Epoch: 23/1000 Iteration: 420 Validat

Epoch: 41/1000 Iteration: 745 Train loss: 0.546689 Train acc: 0.720000
Epoch: 41/1000 Iteration: 750 Train loss: 0.617142 Train acc: 0.880000
Epoch: 41/1000 Iteration: 750 Validation loss: 1.163922 Validation acc: 0.413333
Epoch: 41/1000 Iteration: 755 Train loss: 0.455972 Train acc: 0.960000
Epoch: 42/1000 Iteration: 760 Train loss: 0.659553 Train acc: 0.840000
Epoch: 42/1000 Iteration: 760 Validation loss: 1.164538 Validation acc: 0.406667
Epoch: 42/1000 Iteration: 765 Train loss: 0.627274 Train acc: 0.800000
Epoch: 42/1000 Iteration: 770 Train loss: 0.541843 Train acc: 0.840000
Epoch: 42/1000 Iteration: 770 Validation loss: 1.167063 Validation acc: 0.406667
Epoch: 43/1000 Iteration: 775 Train loss: 0.700970 Train acc: 0.760000
Epoch: 43/1000 Iteration: 780 Train loss: 0.689980 Train acc: 0.720000
Epoch: 43/1000 Iteration: 780 Validation loss: 1.171623 Validation acc: 0.413333
Epoch: 43/1000 Iteration: 785 Train loss: 0.667977 Train acc: 0.760000
Epoch: 43/1000 Iteration: 790 Train l

Epoch: 61/1000 Iteration: 1110 Validation loss: 1.190366 Validation acc: 0.420000
Epoch: 61/1000 Iteration: 1115 Train loss: 0.360930 Train acc: 0.960000
Epoch: 62/1000 Iteration: 1120 Train loss: 0.633475 Train acc: 0.720000
Epoch: 62/1000 Iteration: 1120 Validation loss: 1.194036 Validation acc: 0.420000
Epoch: 62/1000 Iteration: 1125 Train loss: 0.495362 Train acc: 0.880000
Epoch: 62/1000 Iteration: 1130 Train loss: 0.322994 Train acc: 0.960000
Epoch: 62/1000 Iteration: 1130 Validation loss: 1.196029 Validation acc: 0.420000
Epoch: 63/1000 Iteration: 1135 Train loss: 0.536849 Train acc: 0.880000
Epoch: 63/1000 Iteration: 1140 Train loss: 0.327493 Train acc: 0.880000
Epoch: 63/1000 Iteration: 1140 Validation loss: 1.199719 Validation acc: 0.420000
Epoch: 63/1000 Iteration: 1145 Train loss: 0.397878 Train acc: 0.880000
Epoch: 63/1000 Iteration: 1150 Train loss: 0.360253 Train acc: 0.920000
Epoch: 63/1000 Iteration: 1150 Validation loss: 1.197121 Validation acc: 0.420000
Epoch: 64/1000

Epoch: 82/1000 Iteration: 1480 Train loss: 0.265608 Train acc: 0.960000
Epoch: 82/1000 Iteration: 1480 Validation loss: 1.224977 Validation acc: 0.420000
Epoch: 82/1000 Iteration: 1485 Train loss: 0.301808 Train acc: 0.920000
Epoch: 82/1000 Iteration: 1490 Train loss: 0.161013 Train acc: 1.000000
Epoch: 82/1000 Iteration: 1490 Validation loss: 1.227229 Validation acc: 0.420000
Epoch: 83/1000 Iteration: 1495 Train loss: 0.269304 Train acc: 0.960000
Epoch: 83/1000 Iteration: 1500 Train loss: 0.302460 Train acc: 0.960000
Epoch: 83/1000 Iteration: 1500 Validation loss: 1.228785 Validation acc: 0.426667
Epoch: 83/1000 Iteration: 1505 Train loss: 0.285731 Train acc: 0.960000
Epoch: 83/1000 Iteration: 1510 Train loss: 0.262201 Train acc: 0.920000
Epoch: 83/1000 Iteration: 1510 Validation loss: 1.228092 Validation acc: 0.426667
Epoch: 84/1000 Iteration: 1515 Train loss: 0.183907 Train acc: 0.960000
Epoch: 84/1000 Iteration: 1520 Train loss: 0.262710 Train acc: 0.960000
Epoch: 84/1000 Iteration

Epoch: 102/1000 Iteration: 1845 Train loss: 0.179371 Train acc: 1.000000
Epoch: 102/1000 Iteration: 1850 Train loss: 0.172936 Train acc: 0.960000
Epoch: 102/1000 Iteration: 1850 Validation loss: 1.277485 Validation acc: 0.446667
Epoch: 103/1000 Iteration: 1855 Train loss: 0.208621 Train acc: 0.960000


In [ ]:
# Plot training and test loss
t = np.arange(iteration-1)

plt.figure(figsize = (6,6))
plt.plot(t, np.array(train_loss), 'r-', t[t % 10 == 0], np.array(validation_loss), 'b*')
plt.xlabel("iteration")
plt.ylabel("Loss")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
# Plot Accuracies
plt.figure(figsize = (6,6))

plt.plot(t, np.array(train_acc), 'r-', t[t % 10 == 0], validation_acc, 'b*')
plt.xlabel("iteration")
plt.ylabel("Accuray")
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

## Evaluate on test set

In [ ]:
test_acc = []

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints-cnn'))
    
    for x_t, y_t in get_batches(X_test, y_test, batch_size):
        feed = {inputs_: x_t,
                labels_: y_t,
                keep_prob_: 1}
        
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_acc.append(batch_acc)
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))